In [1]:
#Sina Gholami
#MIRCV 2021
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!pip install whoosh

In [3]:
# %matplotlib inline

from whoosh import index
from whoosh.fields import *
from whoosh.analysis import *
from whoosh import qparser
import os, os.path

from matplotlib import pyplot as plt
from IPython.display import display, HTML

from base64 import b64encode
from urllib.parse import quote
from io import BytesIO

BASE_DIR = '/content/gdrive/My Drive/mircv2021'

COCO_CAPTIONS_DATASET = BASE_DIR + '/data/txt/captions.properties'
COCO_ANNOTATIONS_DATASET = BASE_DIR + '/data/txt/annotations.properties'
INDEX_NAME = 'coco'
IMG_DIR = BASE_DIR + '/data/' + 'coco_img'

In [ ]:
#TODO Indexing
schema = Schema(annotations=TEXT(analyzer=KeywordAnalyzer(), vector=True, stored=True), text=TEXT(analyzer=KeywordAnalyzer(), vector=True, stored=True), id=ID(stored=True))

if not os.path.exists(BASE_DIR + "/out"):
    os.mkdir(BASE_DIR + "/out")
elif not os.path.exists(BASE_DIR + "/out/whoosh"):
    os.mkdir(BASE_DIR + "/out/whoosh")

#create whoosh index
ix = index.create_in(BASE_DIR + "/out/prova", schema=schema)
print('creating index...')
#inizialize whoosh writer
writer = ix.writer()
#index all the caption text
with open(COCO_CAPTIONS_DATASET) as captions, open(COCO_ANNOTATIONS_DATASET) as annotations:
    for caption_line, annotation_line in zip(captions, annotations):
        CAPTION_ID, CAPTION_TEXT = caption_line.split("=")
        ANNOTATION_ID, ANNOTATION_TEXT = annotation_line.split("=")
        new_id = CAPTION_ID.zfill(12)+".jpg"
        writer.add_document(id=new_id, text=CAPTION_TEXT, annotations=ANNOTATION_TEXT)
            
  #commit writer
    writer.commit()

print('indexing done')
#commit writer



In [6]:
def get_table(img, title, txt):
    table ='<table><td><img style="width:320px" src="' + img + '"></td>'
    table +='<td style="text-align:left"><b>' + title + '</b></p>' + txt + '</td></table>'
    return table


def display_result(img_path, title, txt):
    with open(img_path, 'rb') as f:
        dataEncoded = b64encode(BytesIO(f.read()).getvalue())
        data = 'data:image/jpg;base64,{}'.format(quote(dataEncoded))
        display(HTML(get_table(data, title, txt)))

In [15]:
#TODO Searching

#txt = "annotations:person text:people text:bike"
#txt = "annotations:person text:people bike"

#txt = "annotations:person text:people NOT text:bike"
txt = "annotations:person text:beach OR text:bird"
#txt = "annotations:person text:people text:bike"


print("query: " + txt)

#initialize whoosh searcher
searcher = ix.searcher()
#perform txt query
parser = qparser.QueryParser("text", ix.schema, group=qparser.AndGroup)
txt_parser = parser.parse(txt)
res = searcher.search(txt_parser)
#display the search results
for hit in res:
  score = hit.score
  id = hit['id']
  text = hit['text']
  display_result(IMG_DIR + '/' + id, id, text)


Output hidden; open in https://colab.research.google.com to view.